In [21]:
import os
import h5py
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from variable_share import y

from sklearn.preprocessing import StandardScaler
from scipy.stats import skew, kurtosis

from sklearn.cluster import MiniBatchKMeans

Sample

In [4]:
path = 'D:/songdata/MillionSongSubset/A/O/G/TRAOGTM128F931FABE.h5'
f = h5py.File(path, 'r')

dict(f['analysis'])

{'bars_confidence': <HDF5 dataset "bars_confidence": shape (1000,), type "<f8">,
 'bars_start': <HDF5 dataset "bars_start": shape (1000,), type "<f8">,
 'beats_confidence': <HDF5 dataset "beats_confidence": shape (1000,), type "<f8">,
 'beats_start': <HDF5 dataset "beats_start": shape (1000,), type "<f8">,
 'sections_confidence': <HDF5 dataset "sections_confidence": shape (14,), type "<f8">,
 'sections_start': <HDF5 dataset "sections_start": shape (14,), type "<f8">,
 'segments_confidence': <HDF5 dataset "segments_confidence": shape (1506,), type "<f8">,
 'segments_loudness_max': <HDF5 dataset "segments_loudness_max": shape (1506,), type "<f8">,
 'segments_loudness_max_time': <HDF5 dataset "segments_loudness_max_time": shape (1506,), type "<f8">,
 'segments_loudness_start': <HDF5 dataset "segments_loudness_start": shape (1506,), type "<f8">,
 'segments_pitches': <HDF5 dataset "segments_pitches": shape (1506, 12), type "<f8">,
 'segments_start': <HDF5 dataset "segments_start": shape (15

# File path

In [5]:
directory = "D:/songdata/MillionSongSubset"

file_paths = []
for dirpath, dirnames, filenames in os.walk(directory):
    for filename in filenames:
        if filename.endswith('.h5'):
            file_paths.append(dirpath+'/'+filename)

file_paths = [file_path.replace('\\', '/') for file_path in file_paths]

In [83]:
directory = "D:/songdata/MillionSongSubset"

def read_from_filepaths(directory, feature_name):
    file_paths = []
    
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith('.h5'):
                file_paths.append(dirpath+'/'+filename)

    file_paths = [file_path.replace('\\', '/') for file_path in file_paths]

    acoustic_feature_lis = []

    for file_path in file_paths:
        file = h5py.File(file_path, 'r')
        acoustic_feature_lis.append(np.array(file['analysis'][feature_name]))
    
    return acoustic_feature_lis

# X_stat matrix

In [9]:
bar_array = []
for file_path in file_paths:
    file = h5py.File(file_path, 'r')
    bar_array.append(np.array(file['analysis']['bars_start']))

In [14]:
def seq_stats(x):
    x = np.asarray(x)
    x = x[np.isfinite(x)]
    if x.size == 0:
        return np.array([np.nan]*10)
    q10, q50, q90 = np.percentile(x, [10,50,90])
    iqr = np.percentile(x, 75) - np.percentile(x, 25)
    cv = np.std(x) / (np.mean(x)+1e-8)
    from scipy.stats import skew, kurtosis
    return np.array([np.mean(x), np.std(x), iqr, cv, skew(x), kurtosis(x), q10, q50, q90, len(x)])


def hist_feat(x, bins=20):
    if len(x)==0: return np.zeros(bins)
    h, _ = np.histogram(x, bins=bins, density=True)
    return h

rows = []
for b in bar_array:  # list[np.ndarray]
    if b is None or len(b)<2:
        rows.append(np.zeros(10+20))  # 占位
        continue
    
    d = np.diff(b)
    d = d[d>0]
    if d.size == 0:
        rows.append(np.zeros(10+20))
        continue
    d = d / (np.median(d)+1e-8)
    rows.append(np.concatenate([seq_stats(d), hist_feat(d, bins=20)])) #总共 30 个 statistics

X_stat = np.vstack(rows)  # (n_songs, ~30)
X_stat = np.nan_to_num(X_stat, nan=0.0, posinf=0.0, neginf=0.0)

In [ ]:
def seq_stats(x):
    x = np.asarray(x)
    x = x[np.isfinite(x)]
    if x.size == 0:
        return np.array([np.nan]*10)
    q10, q50, q90 = np.percentile(x, [10,50,90])
    iqr = np.percentile(x, 75) - np.percentile(x, 25)
    cv = np.std(x) / (np.mean(x)+1e-8)
    from scipy.stats import skew, kurtosis
    return np.array([np.mean(x), np.std(x), iqr, cv, skew(x), kurtosis(x), q10, q50, q90, len(x)])


def hist_feat(x, bins=20):
    if len(x)==0: return np.zeros(bins)
    h, _ = np.histogram(x, bins=bins, density=True)
    return h

def cal_stat_matrix(acoustic_array):
    rows = []

    for b in acoustic_array:  # list[np.ndarray]
        if b is None or len(b)<2:
            rows.append(np.zeros(10+20))  # 占位
            continue
        
        d = np.diff(b)
        d = d[d>0]
        if d.size == 0:
            rows.append(np.zeros(10+20))
            continue
        d = d / (np.median(d)+1e-8)
        rows.append(np.concatenate([seq_stats(d), hist_feat(d, bins=20)])) #总共 30 个 statistics

    X_stat = np.vstack(rows)  # (n_songs, ~30)
    X_stat = np.nan_to_num(X_stat, nan=0.0, posinf=0.0, neginf=0.0)
    return X_stat

(10000, 30)

In [20]:
pd.DataFrame(X_stat).isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
dtype: int64

# Song acoutic feature

## bars start

In [167]:
bar_array = read_from_filepaths(directory, "bars_start")

In [168]:
X_stat = cal_stat_matrix(bar_array)

In [169]:
def build_cluster_oof_features(X_stats, y, k=32, n_splits=5, task="classification"):
    n = len(y)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    oof_dist  = np.zeros((n, k)) # distance to core of each cluster, each song
    oof_mean  = np.zeros((n, 1)) # mean, each song
    oof_size  = np.zeros((n, 1)) # 
    oof_label = np.zeros(n, dtype=int) # cluster labrl, each song

    # 为了在测试时使用：训练完后再用全量拟合一遍
    #scaler_full = StandardScaler().fit(X_stats)
    #Xz_full = scaler_full.transform(X_stats)
    #km_full = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=2048, n_init="auto").fit(Xz_full)

    for tr, va in kf.split(X_stats): # training, validation sets
        # 1) 该折的标准化 + 聚类只在训练折上拟合
        scaler = StandardScaler().fit(X_stats[tr])
        Xz_tr = scaler.transform(X_stats[tr])
        Xz_va = scaler.transform(X_stats[va])

        km = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=2048, n_init="auto").fit(Xz_tr)

        # 2) 给验证折打特征
        oof_dist[va]  = km.transform(Xz_va)        # 到质心距离（越小越近）
        oof_label[va] = km.predict(Xz_va)

        # 3) 只用训练折按簇统计目标
        #lab_tr = km.predict(Xz_tr)
        #mean_by_c = {}  # 每一个cluster 的 mean
        #size_by_c = {} # 每一个cluster 的 size
        #base = y[tr].mean()
        #for c in range(k):
        #    mask = (lab_tr == c) 
        #    size = int(mask.sum())
        #    size_by_c[c] = size #每一个 label 的 size, training set
        #    if size > 0:
        #        m = y[tr][mask].mean() if task == "regression" else y[tr][mask].mean()
        #    else:
        #        m = base
        #    mean_by_c[c] = m

        # 4) 把簇统计赋给验证折样本
        #oof_mean[va, 0] = np.array([mean_by_c[c] for c in oof_label[va]])
        #oof_size[va, 0] = np.array([size_by_c[c] for c in oof_label[va]])

    # 供测试阶段：用全训练集的簇统计
    #lab_full = km_full.predict(Xz_full)
    #full_mean = {c: (y[lab_full==c].mean() if (lab_full==c).sum()>0 else y.mean()) for c in range(k)}
    #full_size = {c: (lab_full==c).sum() for c in range(k)}

    #cluster_stats_full = {"mean": full_mean, "size": full_size}

    return (oof_dist, #oof_mean, oof_size, 
            oof_label,
            #scaler_full, km_full, cluster_stats_full
            )

In [170]:
array_cluster_bars = build_cluster_oof_features(X_stat, y)

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

In [171]:
len(array_cluster_bars)

2

In [172]:
array_cluster_bars[0].shape, array_cluster_bars[1].shape # clustering label predicted from validation sets

((10000, 32), (10000,))

In [173]:
combined_matrix_bars = np.concatenate((array_cluster_bars[0], array_cluster_bars[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_bars.shape

(10000, 33)

## Beat start

In [174]:
beat_array = read_from_filepaths(directory, "beats_start")
X_stat = cal_stat_matrix(beat_array)

In [175]:
array_cluster_beats = build_cluster_oof_features(X_stat, y)

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

In [176]:
combined_matrix_beats = np.concatenate((array_cluster_beats[0], array_cluster_beats[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_beats.shape

(10000, 33)

## Section start

In [179]:
section_array = read_from_filepaths(directory, "sections_start")
X_stat = cal_stat_matrix(section_array)

In [180]:
array_cluster_sections = build_cluster_oof_features(X_stat, y)

combined_matrix_sections = np.concatenate((array_cluster_sections[0], array_cluster_sections[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_sections.shape

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

(10000, 33)

## segments_loudness_max

In [181]:
loudness_array = read_from_filepaths(directory, "segments_loudness_max")
X_stat = cal_stat_matrix(loudness_array)

array_cluster_loudness = build_cluster_oof_features(X_stat, y)

combined_matrix_loudness = np.concatenate((array_cluster_loudness[0], array_cluster_loudness[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_loudness.shape

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

(10000, 33)

## segments_loudness_max_time

In [182]:
loudness_time_array = read_from_filepaths(directory, "segments_loudness_max_time")
X_stat = cal_stat_matrix(loudness_time_array)

array_cluster_loudness_time = build_cluster_oof_features(X_stat, y)

combined_matrix_loudness_time = np.concatenate((array_cluster_loudness_time[0], array_cluster_loudness_time[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_loudness_time.shape

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

(10000, 33)

## segments_loudness_start

In [ ]:
loudness_start_array = read_from_filepaths(directory, "segments_loudness_start")
X_stat = cal_stat_matrix(loudness_start_array)

array_cluster_loudness_start = build_cluster_oof_features(X_stat, y)

combined_matrix_loudness_start = np.concatenate((array_cluster_loudness_start[0], array_cluster_loudness_start[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_loudness_start.shape

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

(10000, 33)

## segments_start

In [184]:
segments_start_array = read_from_filepaths(directory, "segments_start")
X_stat = cal_stat_matrix(segments_start_array)

array_cluster_segments_start = build_cluster_oof_features(X_stat, y)

combined_matrix_segments_start = np.concatenate((array_cluster_segments_start[0], array_cluster_segments_start[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_segments_start.shape

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

(10000, 33)

## tatums_start

In [185]:
tatums_start_array = read_from_filepaths(directory, "tatums_start")
X_stat = cal_stat_matrix(tatums_start_array)

array_cluster_tatums_start = build_cluster_oof_features(X_stat, y)

combined_matrix_tatums_start = np.concatenate((array_cluster_tatums_start[0], array_cluster_tatums_start[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_tatums_start.shape

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

(10000, 33)

# Matrix creeated

In [97]:
combined_matrix_bars.shape, combined_matrix_beats.shape, combined_matrix_sections.shape,combined_matrix_loudness.shape, combined_matrix_loudness_time.shape

((10000, 33), (10000, 33), (10000, 33), (10000, 33), (10000, 33))

In [106]:
all_matrix_combiend = np.concatenate([combined_matrix_bars, combined_matrix_beats, combined_matrix_sections, combined_matrix_loudness, 
                                      combined_matrix_loudness_time
                                      ], axis=1)

In [101]:
all_matrix_combiend.shape

(10000, 165)

In [108]:
np.save("acoustic_features_part.npy", all_matrix_combiend)

In [103]:
scaler = StandardScaler()
all_matrix_combiend = scaler.fit_transform(all_matrix_combiend)

In [105]:
np.save("acoustic_features_standardized_part.npy", all_matrix_combiend)

In [159]:
cluster_label_bars = array_cluster_bars[1].reshape(-1, 1)
cluster_label_beats = array_cluster_beats[1].reshape(-1,1)
cluster_label_loudness = array_cluster_loudness[1].reshape(-1, 1)
cluster_label_loudness_time = array_cluster_loudness_time[1].reshape(-1, 1)

In [160]:
np.save("cluster_label_bars", cluster_label_bars)
np.save("cluster_label_beats", cluster_label_beats)
np.save("cluster_label_loudness", cluster_label_loudness)
np.save("cluster_label_loudness_time", cluster_label_loudness_time)

In [163]:
cluster_distance_bars = array_cluster_bars[0]
cluster_distance_beats = array_cluster_beats[0]
cluster_distance_loudness = array_cluster_loudness[0]
cluster_distance_loudness_time = array_cluster_loudness_time[0]

In [164]:
np.save("cluster_distance_bars", cluster_distance_bars)
np.save("cluster_distance_beats", cluster_distance_beats)
np.save("cluster_distance_loudness", cluster_distance_loudness)
np.save("cluster_distance_loudness_time", cluster_distance_loudness_time)

补后面的acoustic feature label

In [186]:
cluster_label_sections = array_cluster_sections[1].reshape(-1, 1)
cluster_label_loudness_start = array_cluster_loudness_start[1].reshape(-1,1)
cluster_label_segments_start = array_cluster_segments_start[1].reshape(-1, 1)
cluster_label_tatums_start = array_cluster_tatums_start[1].reshape(-1, 1)

In [187]:
np.save("cluster_label_sections", cluster_label_sections)
np.save("cluster_label_loudness_start", cluster_label_loudness_start)
np.save("cluster_label_segments_start", cluster_label_segments_start)
np.save("cluster_label_tatums_start", cluster_label_tatums_start)